In [1]:
import os
import gc
import csv
import argparse
import pickle

import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

from tqdm import tqdm

from dataset import ICUSepsisDataset
from model import SepsisPredictionModel_A1

/Users/snirlugassy/miniconda3/envs/deep/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def calc_accuracy(predicted:np.array, labels:np.array):
    return sum(predicted == labels) / len(labels)

In [14]:
def calc_f1_score(predicted:np.array, labels:np.array):
    tp = int(np.logical_and(labels==1, predicted==1).sum())
    fp = int(np.logical_and(labels==0, predicted==1).sum())
    fn = int(np.logical_and(labels==1, predicted==0).sum())
    tn = int(np.logical_and(labels==0, predicted==0).sum())
    print('tp = ', tp)
    print('fp = ', fp)
    print('fn = ', fn)
    print('tn = ', tn)
    
    return 2 * tp / (2 * tp + fp + fn)

In [2]:
icu_test = ICUSepsisDataset('./data/test')

Found 10000 files in ./data/test


# Model A.1

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SepsisPredictionModel_A1(input_size=len(ICUSepsisDataset.features))
model.to(device)
model.load_state_dict(torch.load('model_a_1.state', map_location=device))

<All keys matched successfully>

In [4]:
y_true = []
y_predict = []

with tqdm(total=len(icu_test)) as pbar:
    for x,y in icu_test:
        pbar.update(1)
        # ignore invalid samples

        
        y = y.to(device)  # squeeze since batch_size=1
        label = int(y[-1])
        N = len(y)

        if x is None:
            y_true.append(label)
            y_predict.append(0)
            continue

        x = x.to(device).unsqueeze(0)

        # Forward pass
        output = model(x).view(N,2)
        prediction = torch.softmax(output, dim=1).argmax(dim=1)

        assert prediction.shape == y.shape

        y_true.append(label)
        y_predict.append(int(prediction[-1]))

100%|██████████| 10000/10000 [02:11<00:00, 76.22it/s]


In [5]:
y_true = np.array(y_true)
y_predict = np.array(y_predict)

In [10]:
print('Model A.1 accuracy:', calc_accuracy(y_predict, y_true))

Model A.1 accuracy: 0.9259


In [15]:
print('Model A.1 F1-score:', calc_f1_score(y_predict, y_true))

tp =  0
fp =  0
fn =  741
tn =  9259
Model A.1 F1-score: 0.0


We observe that model A.1 has very poor performence, without a single True Positive

# Model A.2

In [ ]:
model = SepsisPredictionModel_A2(input_size=len(ICUSepsisDataset.features))
model.to(device)
model.load_state_dict(torch.load('model_a_2.state', map_location=device))

In [ ]:
y_true = []
y_predict = []

with tqdm(total=len(icu_test)) as pbar:
    for x,y in icu_test:
        pbar.update(1)
        # ignore invalid samples

        
        y = y.to(device)  # squeeze since batch_size=1
        label = int(y[-1])
        N = len(y)

        if x is None:
            y_true.append(label)
            y_predict.append(0)
            continue

        x = x.to(device).unsqueeze(0)

        # Forward pass
        output = model(x).view(N,2)
        prediction = torch.softmax(output, dim=1).argmax(dim=1)

        assert prediction.shape == y.shape

        y_true.append(label)
        y_predict.append(int(prediction[-1]))

In [ ]:
y_true = np.array(y_true)
y_predict = np.array(y_predict)

In [ ]:
print('Model A.2 accuracy:', calc_accuracy(y_predict, y_true))

In [ ]:
print('Model A.2 F1-score:', calc_f1_score(y_predict, y_true))

Model A.2 ....